In [1]:
import glob
import os
import yaml
import pandas as pd

In [38]:
def open_sample_sheet(sample_sheet_fp, lanes=False):
    """Read in an IGM sample sheet and return a pandas DF with primary data table"""
    sample_sheet = pd.read_excel(sample_sheet_fp, skiprows = 18, header=1)

    if lanes:
        sample_sheet = sample_sheet.loc[sample_sheet['Lane'].isin(lanes)]

    return(sample_sheet)

In [51]:
def open_sequencing_manifest(manifest_fp, lanes='False', sheetname='Sample Information'):
    """Read in an IGM sequencing manifest and return a pandas DF with primary data table"""
    sample_sheet = pd.read_excel(manifest_fp, sheetname=sheetname, skiprows = 20, header=1)
    
    if lanes:
        sample_sheet = sample_sheet.loc[sample_sheet['lane'].isin(lanes)]
        
    return(sample_sheet)

In [52]:
def get_read(sample, seq_dir, read):
    """Function to pull a given read based on sample name from the reads directory"""
    reads = glob.glob(os.path.join(seq_dir, "{0}_*_{1}_*.fastq.gz".format(sample, read)))
    if len(reads) == 1:
        return(reads[0])
    elif len(reads) > 1:
        raise ValueError('Too many reads found for {0} in {1}:\n'
                         'read_str: {2}'.format(sample, seq_dir, reads))
    elif len(reads) < 1:
        raise ValueError('Too few reads found for {0} in {1}:\n'
                         'read_str: {1}\n'.format(sample, seq_dir, reads))   

In [53]:
def make_sample_dict_pe(sample_sheet, seq_dir,
                        forward = 'R1',
                        reverse = 'R2',
                        adaptor = '$CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa',
                        phred = 'phred33',
                        sample_header = 'Sample_Prefix',
                        sample_name = 'Sample'):

    samples_pe = {'samples_pe': {sample_sheet.loc[x, sample_name]:
                      {'forward': get_read(sample_sheet.loc[x, sample_header], seq_dir, forward),
                       'reverse': get_read(sample_sheet.loc[x, sample_header], seq_dir, reverse),
                       'adaptor': adaptor,
                       'phred': phred
                      } for x in sample_sheet.index
                 }}
    
    return(samples_pe)

In [54]:
def make_sample_dict_se(sample_sheet, seq_dir,
                        forward = 'R1',
                        adaptor = '$CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa',
                        phred = 'phred33',
                        sample_header = 'Sample_Prefix',
                        sample_name = 'Sample'):

    samples_se = {'samples_se': {sample_sheet.loc[x, sample_name]:
                      {'forward': get_read(sample_sheet.loc[x, sample_header], seq_dir, forward),
                       'adaptor': adaptor,
                       'phred': phred
                      } for x in sample_sheet.index
                 }}
    
    return(samples_se)

In [55]:
def format_yaml_pe(RUN, samples_pe,
                   TMP_DIR_ROOT = '/localscratch',
                   trimmomatic_path = '$CONDA_ENV_PATH/share/trimmomatic-*',
                   adaptor = '$CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa',
                   kneaddata_db = '$CONDA_ENV_PATH/share/kd_dbs/demo',
                   gzip_path = 'gzip',
                   knead_env = 'source activate kneaddata',
                   params = {'trimmomatic_params': 'LEADING:20 TRAILING:20 AVGQUAL:30 MINLEN:32 TOPHRED33'},
                   default_flow_style = False):
    
    config_str = ''

    config_str += yaml.dump({'TMP_DIR_ROOT': TMP_DIR_ROOT}, default_flow_style = default_flow_style)

    config_str += yaml.dump({'RUN': RUN}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump({'KNEAD_ENV': knead_env}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump({'kneaddata_db': kneaddata_db}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump(samples_pe, default_flow_style = default_flow_style)

    config_str += yaml.dump({'software': 
                   {'trimmomatic': trimmomatic_path,
                    'gzip': gzip_path}
              }, default_flow_style = default_flow_style)

    config_str += yaml.dump(params,
                            default_flow_style = default_flow_style)

    return(config_str)

In [56]:
def format_yaml_se(RUN, samples_se,
                   TMP_DIR_ROOT = '/localscratch',
                   trimmomatic_path = '$CONDA_ENV_PATH/share/trimmomatic-*',
                   adaptor = '$CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa',
                   kneaddata_db = '$CONDA_ENV_PATH/share/kd_dbs/demo',
                   gzip_path = 'gzip',
                   knead_env = 'source activate kneaddata',
                   params = {'trimmomatic_params': 'LEADING:20 TRAILING:20 AVGQUAL:30 MINLEN:32 TOPHRED33'},
                   default_flow_style = False):
    
    config_str = ''

    config_str += yaml.dump({'TMP_DIR_ROOT': TMP_DIR_ROOT}, default_flow_style = default_flow_style)

    config_str += yaml.dump({'RUN': RUN}, default_flow_style = default_flow_style)

    config_str += yaml.dump({'KNEAD_ENV': knead_env}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump({'kneaddata_db': kneaddata_db}, default_flow_style = default_flow_style)
    
    config_str += yaml.dump(samples_se, default_flow_style = default_flow_style)

    config_str += yaml.dump({'software': 
                   {'trimmomatic': trimmomatic_path,
                    'gzip': gzip_path}
              }, default_flow_style = default_flow_style)

    config_str += yaml.dump(params,
                            default_flow_style = default_flow_style)

    return(config_str)

In [57]:
sample_sheet_fp = './example/reads/example_sample_sheet.xls'
lanes = [1]
seq_dir = './example/reads/Run1'

sample_sheet = open_sample_sheet(sample_sheet_fp, lanes=lanes)


# These columns can be modified with regexes to fix sample names or add common sample prefixes if necessary
sample_sheet['Sample_Prefix'] = sample_sheet['Sample_ID']
sample_sheet['Sample'] = sample_sheet['Sample_ID']


sample_sheet

,Lane,Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,Sample_Project,Description,Sample_Prefix,Sample
0,1,sample1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sample1,sample1
1,1,sample2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,sample2,sample2


Make samples dictionary
------

This links the sample name to the original location of the forward (and reverse) reads. 

In [58]:
samples_pe = make_sample_dict_pe(sample_sheet, seq_dir,
                        adaptor = '$CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa',
                        forward = 'R1',
                        reverse = 'R2',
                        sample_header = 'Sample_Prefix',
                        sample_name = 'Sample')

Format YAML file
================

Make the actual file.

Going to want to modify `RUN` variable with a good name for the run.

In [59]:
RUN = 'example_run'
config_str = format_yaml_pe(RUN, samples_pe)
with open('config_%s.yaml' % RUN, 'w') as f:
    f.write(config_str)

In [60]:
print(config_str)

TMP_DIR_ROOT: /localscratch
RUN: example_run
KNEAD_ENV: source activate kneaddata
kneaddata_db: $CONDA_ENV_PATH/share/kd_dbs/demo
samples_pe:
  sample1:
    adaptor: $CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa
    forward: ./example/reads/Run1/sample1_S312_R1_L001.fastq.gz
    phred: phred33
    reverse: ./example/reads/Run1/sample1_S312_R2_L001.fastq.gz
  sample2:
    adaptor: $CONDA_ENV_PATH/share/trimmomatic-*/adapters/TruSeq3-PE-2.fa
    forward: ./example/reads/Run1/sample2_S521_R1_L001.fastq.gz
    phred: phred33
    reverse: ./example/reads/Run1/sample2_S521_R2_L001.fastq.gz
software:
  gzip: gzip
  trimmomatic: $CONDA_ENV_PATH/share/trimmomatic-*
trimmomatic_params: LEADING:20 TRAILING:20 AVGQUAL:30 MINLEN:32 TOPHRED33

